# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** The architecture of the CNN was given as a pre-trained model and at the end of it, there is an embedding layer that convert our dimensionality and that is a learnable layer to addapte the pre-trained network to our purpose. The RNN architecture is consist of an embedding layer and LSTM layer. The number of LSTM layer is 1 as notebook's default value that was given by you. I used the suggested hyperparameter values in the lesson. I did not increase the number of epochs because the training process taking too much time. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The images were resized 224x224 because of our pretrained model(that we used in the encoder part)'s input dimensions. The normalization values are according to the pretrained model and all pretrained models in torchvision have the same normalization (same mean and std values). 


### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I did not update the pretrained model's parameters because it has already trained on a dataset and I add an embedding layer. The purpose of the embedding layer in the CNN(encoder) part is to addapte the pre-trained network to our purpose. The RNN part is the one that we created and trained completely.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Adam optimizer is the most reccomended optimizer and I start with it. The results in the next part was straightforward and I did not change it.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 800           # dimensionality of image and word embeddings
hidden_size = 512         # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 10          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.85s)
creating index...


  0%|          | 185/414113 [00:00<03:44, 1844.98it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:32<00:00, 4492.90it/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:00<00:00, 103431290.67it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [10/6471], Loss: 5.2365, Perplexity: 188.01801
Epoch [1/3], Step [20/6471], Loss: 4.7290, Perplexity: 113.1861
Epoch [1/3], Step [30/6471], Loss: 4.3678, Perplexity: 78.87090
Epoch [1/3], Step [40/6471], Loss: 4.0916, Perplexity: 59.8384
Epoch [1/3], Step [50/6471], Loss: 4.0875, Perplexity: 59.5913
Epoch [1/3], Step [60/6471], Loss: 3.9643, Perplexity: 52.6853
Epoch [1/3], Step [70/6471], Loss: 3.5704, Perplexity: 35.5317
Epoch [1/3], Step [80/6471], Loss: 3.6290, Perplexity: 37.6753
Epoch [1/3], Step [90/6471], Loss: 3.6422, Perplexity: 38.1752
Epoch [1/3], Step [100/6471], Loss: 3.5920, Perplexity: 36.3069
Epoch [1/3], Step [110/6471], Loss: 3.5418, Perplexity: 34.5288
Epoch [1/3], Step [120/6471], Loss: 3.5552, Perplexity: 34.9947
Epoch [1/3], Step [130/6471], Loss: 3.5275, Perplexity: 34.0372
Epoch [1/3], Step [140/6471], Loss: 3.6969, Perplexity: 40.3219
Epoch [1/3], Step [150/6471], Loss: 3.5104, Perplexity: 33.4608
Epoch [1/3], Step [160/6471], Loss: 3.2769, P

Epoch [1/3], Step [2540/6471], Loss: 2.4657, Perplexity: 11.7716
Epoch [1/3], Step [2550/6471], Loss: 2.3291, Perplexity: 10.2687
Epoch [1/3], Step [2560/6471], Loss: 2.7943, Perplexity: 16.3520
Epoch [1/3], Step [2570/6471], Loss: 2.4877, Perplexity: 12.0338
Epoch [1/3], Step [2580/6471], Loss: 2.1846, Perplexity: 8.88679
Epoch [1/3], Step [2590/6471], Loss: 2.4802, Perplexity: 11.9435
Epoch [1/3], Step [2600/6471], Loss: 2.3746, Perplexity: 10.7471
Epoch [1/3], Step [2610/6471], Loss: 2.7177, Perplexity: 15.1460
Epoch [1/3], Step [2620/6471], Loss: 2.3412, Perplexity: 10.3942
Epoch [1/3], Step [2630/6471], Loss: 2.3012, Perplexity: 9.98633
Epoch [1/3], Step [2640/6471], Loss: 2.4246, Perplexity: 11.2976
Epoch [1/3], Step [2650/6471], Loss: 2.4668, Perplexity: 11.7847
Epoch [1/3], Step [2660/6471], Loss: 2.3483, Perplexity: 10.4679
Epoch [1/3], Step [2670/6471], Loss: 2.1943, Perplexity: 8.97369
Epoch [1/3], Step [2680/6471], Loss: 2.3152, Perplexity: 10.1273
Epoch [1/3], Step [2690/6

Epoch [1/3], Step [5080/6471], Loss: 2.0906, Perplexity: 8.08979
Epoch [1/3], Step [5090/6471], Loss: 2.2758, Perplexity: 9.73610
Epoch [1/3], Step [5100/6471], Loss: 2.3978, Perplexity: 10.9989
Epoch [1/3], Step [5120/6471], Loss: 2.0787, Perplexity: 7.99444
Epoch [1/3], Step [5130/6471], Loss: 3.1460, Perplexity: 23.2436
Epoch [1/3], Step [5140/6471], Loss: 2.1303, Perplexity: 8.41777
Epoch [1/3], Step [5150/6471], Loss: 2.2297, Perplexity: 9.29694
Epoch [1/3], Step [5160/6471], Loss: 2.1246, Perplexity: 8.36993
Epoch [1/3], Step [5170/6471], Loss: 2.5225, Perplexity: 12.4601
Epoch [1/3], Step [5180/6471], Loss: 2.0552, Perplexity: 7.80870
Epoch [1/3], Step [5190/6471], Loss: 2.0132, Perplexity: 7.48752
Epoch [1/3], Step [5200/6471], Loss: 2.1077, Perplexity: 8.22907
Epoch [1/3], Step [5210/6471], Loss: 2.4201, Perplexity: 11.2473
Epoch [1/3], Step [5220/6471], Loss: 2.4272, Perplexity: 11.3266
Epoch [1/3], Step [5230/6471], Loss: 2.5384, Perplexity: 12.6591
Epoch [1/3], Step [5240/6

Epoch [2/3], Step [1150/6471], Loss: 2.4608, Perplexity: 11.7137
Epoch [2/3], Step [1160/6471], Loss: 2.0901, Perplexity: 8.08613
Epoch [2/3], Step [1170/6471], Loss: 2.0822, Perplexity: 8.02253
Epoch [2/3], Step [1180/6471], Loss: 2.3325, Perplexity: 10.3032
Epoch [2/3], Step [1190/6471], Loss: 2.4536, Perplexity: 11.6306
Epoch [2/3], Step [1200/6471], Loss: 1.9695, Perplexity: 7.16673
Epoch [2/3], Step [1210/6471], Loss: 2.2360, Perplexity: 9.35591
Epoch [2/3], Step [1220/6471], Loss: 2.0586, Perplexity: 7.83545
Epoch [2/3], Step [1230/6471], Loss: 2.2384, Perplexity: 9.37806
Epoch [2/3], Step [1240/6471], Loss: 2.1266, Perplexity: 8.38667
Epoch [2/3], Step [1250/6471], Loss: 2.0715, Perplexity: 7.93691
Epoch [2/3], Step [1260/6471], Loss: 2.2723, Perplexity: 9.70139
Epoch [2/3], Step [1270/6471], Loss: 2.0800, Perplexity: 8.00434
Epoch [2/3], Step [1280/6471], Loss: 2.0976, Perplexity: 8.14664
Epoch [2/3], Step [1290/6471], Loss: 2.2599, Perplexity: 9.5823
Epoch [2/3], Step [1300/64

Epoch [2/3], Step [3670/6471], Loss: 2.1286, Perplexity: 8.40349
Epoch [2/3], Step [3680/6471], Loss: 2.1623, Perplexity: 8.6914
Epoch [2/3], Step [3690/6471], Loss: 1.9643, Perplexity: 7.12960
Epoch [2/3], Step [3700/6471], Loss: 2.2409, Perplexity: 9.40226
Epoch [2/3], Step [3710/6471], Loss: 1.9936, Perplexity: 7.34206
Epoch [2/3], Step [3720/6471], Loss: 2.5099, Perplexity: 12.3032
Epoch [2/3], Step [3730/6471], Loss: 2.2513, Perplexity: 9.49998
Epoch [2/3], Step [3740/6471], Loss: 1.9960, Perplexity: 7.35935
Epoch [2/3], Step [3750/6471], Loss: 2.1635, Perplexity: 8.7019
Epoch [2/3], Step [3760/6471], Loss: 1.9814, Perplexity: 7.25305
Epoch [2/3], Step [3770/6471], Loss: 2.5575, Perplexity: 12.9032
Epoch [2/3], Step [3780/6471], Loss: 1.9252, Perplexity: 6.85651
Epoch [2/3], Step [3790/6471], Loss: 1.9968, Perplexity: 7.36516
Epoch [2/3], Step [3800/6471], Loss: 2.6904, Perplexity: 14.7370
Epoch [2/3], Step [3810/6471], Loss: 2.0385, Perplexity: 7.67905
Epoch [2/3], Step [3820/647

Epoch [2/3], Step [6190/6471], Loss: 1.9999, Perplexity: 7.38810
Epoch [2/3], Step [6200/6471], Loss: 1.9628, Perplexity: 7.11908
Epoch [2/3], Step [6210/6471], Loss: 2.6422, Perplexity: 14.0447
Epoch [2/3], Step [6220/6471], Loss: 2.2769, Perplexity: 9.74665
Epoch [2/3], Step [6230/6471], Loss: 2.1137, Perplexity: 8.27919
Epoch [2/3], Step [6240/6471], Loss: 1.9205, Perplexity: 6.8245
Epoch [2/3], Step [6250/6471], Loss: 2.2510, Perplexity: 9.4970
Epoch [2/3], Step [6260/6471], Loss: 1.9313, Perplexity: 6.8987
Epoch [2/3], Step [6270/6471], Loss: 2.0717, Perplexity: 7.93831
Epoch [2/3], Step [6280/6471], Loss: 3.0142, Perplexity: 20.3734
Epoch [2/3], Step [6290/6471], Loss: 2.2255, Perplexity: 9.2586
Epoch [2/3], Step [6300/6471], Loss: 2.8951, Perplexity: 18.0849
Epoch [2/3], Step [6310/6471], Loss: 1.9692, Perplexity: 7.16528
Epoch [2/3], Step [6320/6471], Loss: 1.8983, Perplexity: 6.67455
Epoch [2/3], Step [6330/6471], Loss: 2.0690, Perplexity: 7.91734
Epoch [2/3], Step [6340/6471]

Epoch [3/3], Step [2260/6471], Loss: 1.9838, Perplexity: 7.2705
Epoch [3/3], Step [2270/6471], Loss: 1.9591, Perplexity: 7.09285
Epoch [3/3], Step [2280/6471], Loss: 2.2550, Perplexity: 9.53500
Epoch [3/3], Step [2290/6471], Loss: 2.0316, Perplexity: 7.62658
Epoch [3/3], Step [2300/6471], Loss: 2.9876, Perplexity: 19.8382
Epoch [3/3], Step [2310/6471], Loss: 2.1518, Perplexity: 8.60079
Epoch [3/3], Step [2320/6471], Loss: 1.9810, Perplexity: 7.24973
Epoch [3/3], Step [2330/6471], Loss: 2.1288, Perplexity: 8.40489
Epoch [3/3], Step [2340/6471], Loss: 1.9533, Perplexity: 7.0517
Epoch [3/3], Step [2350/6471], Loss: 2.0219, Perplexity: 7.55296
Epoch [3/3], Step [2360/6471], Loss: 2.1188, Perplexity: 8.3209
Epoch [3/3], Step [2370/6471], Loss: 2.0040, Perplexity: 7.4186
Epoch [3/3], Step [2380/6471], Loss: 2.1421, Perplexity: 8.51710
Epoch [3/3], Step [2390/6471], Loss: 2.5461, Perplexity: 12.7573
Epoch [3/3], Step [2400/6471], Loss: 2.1275, Perplexity: 8.39424
Epoch [3/3], Step [2410/6471]

Epoch [3/3], Step [4780/6471], Loss: 2.4536, Perplexity: 11.6305
Epoch [3/3], Step [4790/6471], Loss: 1.8349, Perplexity: 6.26435
Epoch [3/3], Step [4800/6471], Loss: 2.1260, Perplexity: 8.3813
Epoch [3/3], Step [4810/6471], Loss: 2.0190, Perplexity: 7.5310
Epoch [3/3], Step [4820/6471], Loss: 1.8952, Perplexity: 6.65397
Epoch [3/3], Step [4830/6471], Loss: 1.7893, Perplexity: 5.9853
Epoch [3/3], Step [4840/6471], Loss: 2.1586, Perplexity: 8.6588
Epoch [3/3], Step [4850/6471], Loss: 2.0098, Perplexity: 7.4621
Epoch [3/3], Step [4860/6471], Loss: 2.0418, Perplexity: 7.7046
Epoch [3/3], Step [4870/6471], Loss: 2.1330, Perplexity: 8.4404
Epoch [3/3], Step [4880/6471], Loss: 1.9623, Perplexity: 7.1159
Epoch [3/3], Step [4890/6471], Loss: 2.1451, Perplexity: 8.54310
Epoch [3/3], Step [4900/6471], Loss: 2.0351, Perplexity: 7.65318
Epoch [3/3], Step [4910/6471], Loss: 1.8307, Perplexity: 6.2382
Epoch [3/3], Step [4920/6471], Loss: 2.0025, Perplexity: 7.40765
Epoch [3/3], Step [4930/6471], Los

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.